# BLACK LIVES MATTER DATA ANALYSIS
### Data Engineering Capstone Project

#### Project Summary
The purpose of this project is to understand the situation of police-related violence in the United States by mapping up raw data sources from multiple channels.

The dataset currently includes these types of data:

1. Police shootings
2. Police officer deaths suffered in the line of duty
3. City population
4. Locations for all BLM protests since May 25, 2020
5. Crimes statistics in New York-Newark-Jersey City, NY-NJ-PA 


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import configparser
from datetime import datetime
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month,dayofweek, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import *
import logging
import s3fs

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']


### Step 1: Scope the Project and Gather Data

#### Scope 
I am planning to find some raw data from different data sources in regards to victims of police violence, police deaths while on duty, population of cities and crimes statistics in major cities, such as New York. What I want to understand is the relationship between crime rates and police violence, and whether police violence happens more often in certain areas in major cities. I will assess the raw data and see if there are information I can extract to build some dimensional tables. I will focus on New York to start with, then will find more crime statistics in other major cities in the future. 

**Choice of tools:**
- *S3 buckets*:
    - Given that I am intending to gather a lot more bulky data in the future, I decided to use S3 buckets for storage as it is a more economical option. 
- *PySpark*: 
    - I will use Spark cluster to handle big data. Some datasets consist of 5 millions rows and pyspark is a great tool to handle big data. 

#### Describe and Gather Data 
I found this dataset on Kaggle (https://www.kaggle.com/jpmiller/police-violence-in-the-us). The author posted this dataset for data analysts and scientists to explore and find insights. Most datasets mainly come from governmental websites and I will explain each dataset below in details.

### 1. Names of states in the US
- The dataset below shows the name and abbreviation of states in the US.  

In [3]:
# Read in the data here
us_state = "s3://datalake-blm/raw/us_state.csv"

In [4]:
us_state_df = pd.read_csv(us_state)

In [5]:
us_state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state         50 non-null     object
 1   state_abbrev  50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes


In [6]:
us_state_df.head(5)

,state,state_abbrev
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [7]:
us_state_df.count()

state           50
state_abbrev    50
dtype: int64

### 2. Budgets in different states in the US 
- Fiscally standardized finances for 150 large US cities across 120 categories.
- Source: Lincoln Institute of Land Policy, https://www.lincolninst.edu/research-data/data-toolkits/fiscally-standardized-cities

In [8]:
budgets = "s3://datalake-blm/raw/budgets.csv"

In [9]:
budgets_df = pd.read_csv(budgets)

In [10]:
budgets_df.head(5)

,year,city_name,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,igr_state_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
0,1977,AK: Anchorage,22002001.0,174500,4.044885,5342.24,4956.92,2148.77,279.32,1869.46,...,178.51,787.93,691.32,NaN,NaN,NaN,4.0,36855.0,NaN,1.0
1,1978,AK: Anchorage,22002001.0,177000,3.759509,5948.99,5490.05,2468.11,403.24,2064.86,...,187.53,1395.82,1158.01,NaN,NaN,NaN,4.0,36804.0,NaN,1.0
2,1979,AK: Anchorage,22002001.0,179600,3.376308,6158.68,5746.64,2573.34,496.97,2076.37,...,249.03,1812.04,1314.62,NaN,NaN,NaN,4.0,36757.0,NaN,1.0
3,1980,AK: Anchorage,22002001.0,178800,2.974757,5654.93,5210.77,2313.62,371.46,1942.16,...,986.73,1330.72,1572.66,NaN,NaN,NaN,4.0,36008.0,NaN,1.0
4,1981,AK: Anchorage,22002001.0,174431,2.696590,6192.83,5736.81,2771.43,338.76,2432.67,...,977.32,1549.51,1669.55,NaN,NaN,NaN,4.0,34557.0,NaN,1.0


In [11]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6232 entries, 0 to 6231
Columns: 662 entries, year to consolidated_govt
dtypes: float64(658), int64(2), object(2)
memory usage: 31.5+ MB


- This dataset has has more than 600 columns and it is not easy to inspect all the columns. 
- I might just extract the population of each city so I can map this information up with other data sets

In [12]:
budgets_df.count()

year                        6232
city_name                   6232
id_city                     6191
city_population             6232
cpi                         6150
                            ... 
county_population           5418
relationship_city_school    6150
enrollment                  6196
districts_in_city           4741
consolidated_govt           6150
Length: 662, dtype: int64

### 3. Police Killings 
- The dataset below shows descriptive statistics  of civilians killed by police in the US
- Source: Mapping Police Violence, https://mappingpoliceviolence.org/aboutthedata

In [13]:
police_killings = 's3://datalake-blm/raw/police_killings.csv'

In [14]:
police_killings_df = pd.read_csv(police_killings)

In [15]:
police_killings_df.head(5)

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,31/12/2019,Broad St.,Globe,AZ,85501.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name withheld by police,NaN,Male,Unknown race,NaN,31/12/2019,7239-7411 I-40,Memphis,AR,38103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Terry Hudson,57,Male,Black,NaN,31/12/2019,3600 N 24th St,Omaha,NE,68110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Malik Williams,23,Male,Black,NaN,31/12/2019,30800 14th Avenue South,Federal Way,WA,98003.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Frederick Perkins,37,Male,Black,NaN,31/12/2019,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
police_killings_df.nunique()

Victim's name             7411
Victim's age                87
Victim's gender              4
Victim's race                8
URL of image of victim    4191
                          ... 
Unnamed: 61                  0
Unnamed: 62                  0
Unnamed: 63                  0
Unnamed: 64                  0
Unnamed: 65                  0
Length: 66, dtype: int64

In [17]:
police_killings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7907 entries, 0 to 7906
Data columns (total 66 columns):
 #   Column                                                                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                                                                      --------------  -----  
 0   Victim's name                                                                                                                                                               7663 non-null   object 
 1   Victim's age                                                                                                                                                                7596 non-null   object 
 2   Victim's gender                                                                     

- This dataset has so many unnamed columns that have no values. I will need to delete them.
- Some columns have null values. Need to explore that a bit more
- I want to extract the race of the victims and create a dataset with just racial information

### 4. Police Deaths
- The dataset below shows information of police who were killed when on duty 
- Source: FiveThirtyEight, https://github.com/fivethirtyeight/data/tree/master/police-deaths

In [18]:
police_deaths = 's3://datalake-blm/raw/police_deaths_538.csv'

In [19]:
police_deaths_df = pd.read_csv(police_deaths)

In [20]:
police_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   person       22800 non-null  object
 1   dept         22800 non-null  object
 2   eow          22800 non-null  object
 3   cause        22800 non-null  object
 4   cause_short  22800 non-null  object
 5   date         22800 non-null  object
 6   year         22800 non-null  int64 
 7   canine       22800 non-null  bool  
 8   dept_name    22800 non-null  object
 9   state        22800 non-null  object
dtypes: bool(1), int64(1), object(8)
memory usage: 1.6+ MB


- This datasets looks more clean. It doesn't need a lot of further curation

In [21]:
police_deaths_df.head()

,person,dept,eow,cause,cause_short,date,year,canine,dept_name,state
0,Constable Darius Quimby,"Albany County Constable's Office, NY","EOW: Monday, January 3, 1791",Cause of Death: Gunfire,Gunfire,1791-01-03,1791,False,Albany County Constable's Office,NY
1,Sheriff Cornelius Hogeboom,"Columbia County Sheriff's Office, NY","EOW: Saturday, October 22, 1791",Cause of Death: Gunfire,Gunfire,1791-10-22,1791,False,Columbia County Sheriff's Office,NY
2,Deputy Sheriff Isaac Smith,"Westchester County Sheriff's Department, NY","EOW: Thursday, May 17, 1792",Cause of Death: Gunfire,Gunfire,1792-05-17,1792,False,Westchester County Sheriff's Department,NY
3,Marshal Robert Forsyth,United States Department of Justice - United S...,"EOW: Saturday, January 11, 1794",Cause of Death: Gunfire,Gunfire,1794-01-11,1794,False,United States Department of Justice - United S...,US
4,Sheriff Robert Maxwell,"Greenville County Sheriff's Office, SC","EOW: Sunday, November 12, 1797",Cause of Death: Gunfire,Gunfire,1797-11-12,1797,False,Greenville County Sheriff's Office,SC


In [22]:
police_deaths_df.nunique()
## Some names seem to have been duplicated. Will need to make sure I don't de-dupe important information

person         22742
dept            6528
eow            17158
cause             36
cause_short       36
date           17158
year             202
canine             2
dept_name       5525
state             60
dtype: int64

### 5. Protest data
- The dataset below shows all BLM protests since 25th May 2020

In [23]:
protest = 's3a://datalake-blm/raw/protests.json'

Given it is a json file, I wanna use spark to explore it and print the schema

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

In [25]:
protests_df = spark.read.json(protest)

In [26]:
protests_df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- geometry: struct (nullable = true)
 |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- properties: struct (nullable = true)
 |    |    |    |-- Alias: string (nullable = true)
 |    |    |    |-- Batch: string (nullable = true)
 |    |    |    |-- City: string (nullable = true)
 |    |    |    |-- Comments: string (nullable = true)
 |    |    |    |-- Country: string (nullable = true)
 |    |    |    |-- Date_Added: long (nullable = true)
 |    |    |    |-- Mailing_Abbreviation: string (nullable = true)
 |    |    |    |-- OBJECTID: long (nullable = true)
 |    |    |    |-- Region: string (nullable = true)
 |    |    |    |-- Search_Label: string (nullable = true)
 |    |    |    |-- X_Longitude: double (n

- Given that this is a json file, I will need to flatten the columns to turn data into tabular format

In [27]:
protests_df.show(5)

+--------------------+-----------------+
|            features|             type|
+--------------------+-----------------+
|[[[[-134.40678999...|FeatureCollection|
+--------------------+-----------------+



### 6. New York Crime Data
- Crimes Statistics in New York between 2006 and 2019
- City of New York Open Data Sources. Source https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u

In [28]:
ny = 's3a://datalake-blm/raw/NYPD_Arrests_Data__Historic_.csv'

In [29]:
#ny_df = spark.read.json(ny)
ny_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(ny)

In [30]:
ny_df.show(2)

+----------+-----------+-----+--------------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|     OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|          Latitude|         Longitude|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
| 190294601| 11/23/2018|  109|ASSAULT 2,1,UNCLA...|  106|FELONY ASSAULT|PL 1200512|         F|          B|             40|                0|    25-44|       M|         BLACK| 1008096.0|  233595.0|40.807816227000046|-73.9138626609999

In [31]:
ny_df.printSchema()

root
 |-- ARREST_KEY: integer (nullable = true)
 |-- ARREST_DATE: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: integer (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- X_COORD_CD: double (nullable = true)
 |-- Y_COORD_CD: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lon_Lat: string (nullable = true)



In [32]:
ny_df.count()

5012956

### Step 2: Explore and Assess the Data

#### In this section, I will assess and explore the datasets in depth, remove duplicated values and unuseful information

#### 1. State
- Given that the state dataset has unique value and is clean, I will simply will rename column, reorder columns, and save it directly on s3 Stage bucket

In [33]:
# I will use the stage budget on s3 to store all the clean datasets 
s3path = 's3://datalake-blm/stage/'

In [34]:
us_state_df.head(4)

,state,state_abbrev
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR


In [35]:
us_state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state         50 non-null     object
 1   state_abbrev  50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes


In [36]:
us_state_df=us_state_df.rename(columns={"state_abbrev":"state_id"})

In [37]:
us_state_df.head(2)

,state,state_id
0,Alabama,AL
1,Alaska,AK


In [38]:
# I want to rearrange the order of columns 
us_state_df = us_state_df.reindex(columns=['state_id',"state"])

In [39]:
# I need to ensure I remove all the whitespace for each record
us_state_df['state']=us_state_df['state'].str.strip()

In [40]:
state_file = 'state.csv'

In [226]:
us_state_df.to_csv(os.path.join(s3path, state_file),header=True, index = False)

#### 2. Budgets 
- I want to extract the city_population column from this table and create a new column labelling the state of each record

In [42]:
budgets_df.head(2)

,year,city_name,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,igr_state_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
0,1977,AK: Anchorage,22002001.0,174500,4.044885,5342.24,4956.92,2148.77,279.32,1869.46,...,178.51,787.93,691.32,NaN,NaN,NaN,4.0,36855.0,NaN,1.0
1,1978,AK: Anchorage,22002001.0,177000,3.759509,5948.99,5490.05,2468.11,403.24,2064.86,...,187.53,1395.82,1158.01,NaN,NaN,NaN,4.0,36804.0,NaN,1.0


In [43]:
budgets_df.count()

year                        6232
city_name                   6232
id_city                     6191
city_population             6232
cpi                         6150
                            ... 
county_population           5418
relationship_city_school    6150
enrollment                  6196
districts_in_city           4741
consolidated_govt           6150
Length: 662, dtype: int64

- Given that year, city_name and city_population all have the same number of records, I should simply check if there are any duplicates.

In [44]:
len(budgets_df)

6232

In [45]:
budgets_df.nunique()

year                          41
city_name                    152
id_city                      151
city_population             6145
cpi                           41
                            ... 
county_population           4995
relationship_city_school       5
enrollment                  5108
districts_in_city             22
consolidated_govt              2
Length: 662, dtype: int64

In [46]:
budgets_df =budgets_df.drop_duplicates()

In [47]:
len(budgets_df)

6232

- There are no duplicated values in this dataset, thats great! 

In [48]:
#I previously noticed that some records don't have id_city. I want to know what kind information do these records hold
budgets_df[budgets_df.id_city.isnull()].head(3)

,year,city_name,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,igr_state_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
410,1977,Average for Cities,NaN,350083,NaN,2312.35,1889.98,760.76,338.25,422.51,...,366.03,387.54,839.45,NaN,NaN,690459.0,NaN,59844.0,NaN,NaN
411,1978,Average for Cities,NaN,349355,NaN,2290.69,1846.08,696.37,360.90,335.47,...,495.97,403.98,869.54,NaN,NaN,696845.0,NaN,64642.0,NaN,NaN
412,1979,Average for Cities,NaN,348967,NaN,2256.35,1816.53,693.98,352.31,341.68,...,302.54,461.02,893.65,NaN,NaN,704711.0,NaN,56383.0,NaN,NaN


In [49]:
len(budgets_df[budgets_df.id_city.isnull()])

41

- Okay, those records show data average for cities. I don't need them and I will drop these 41 records.

In [50]:
budgets_df_cleaned = budgets_df[budgets_df.id_city.notnull()]

In [51]:
budgets_df_cleaned.count()

year                        6191
city_name                   6191
id_city                     6191
city_population             6191
cpi                         6150
                            ... 
county_population           5377
relationship_city_school    6150
enrollment                  6155
districts_in_city           4741
consolidated_govt           6150
Length: 662, dtype: int64

- Data looks a lot cleaner now. I don't want to use the dropna function because it looks like the issues come from some missing data in te county_name, id_county, enrollment, consolidated_govt and districts_in_city columns. I won't use these columns anyway.

In [52]:
budgets_df_cleaned.head(1)

,year,city_name,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,igr_state_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
0,1977,AK: Anchorage,22002001.0,174500,4.044885,5342.24,4956.92,2148.77,279.32,1869.46,...,178.51,787.93,691.32,NaN,NaN,NaN,4.0,36855.0,NaN,1.0


In [53]:
len(budgets_df_cleaned)

6191

- I want to drop the index column as that column shows inaccurate values

In [54]:
budgets_df_cleaned =budgets_df_cleaned.reset_index(drop=True)

In [55]:
budgets_df_cleaned.head(2)

,year,city_name,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,igr_state_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
0,1977,AK: Anchorage,22002001.0,174500,4.044885,5342.24,4956.92,2148.77,279.32,1869.46,...,178.51,787.93,691.32,NaN,NaN,NaN,4.0,36855.0,NaN,1.0
1,1978,AK: Anchorage,22002001.0,177000,3.759509,5948.99,5490.05,2468.11,403.24,2064.86,...,187.53,1395.82,1158.01,NaN,NaN,NaN,4.0,36804.0,NaN,1.0


- Now I am going to create a dataset with the popultion information only

In [56]:
city_population = budgets_df_cleaned[['year','city_name','id_city','city_population']]

In [57]:
city_population.head(2)

,year,city_name,id_city,city_population
0,1977,AK: Anchorage,22002001.0,174500
1,1978,AK: Anchorage,22002001.0,177000


- I will extract the state information from the city_name column

In [58]:
city_population["state"]= city_population["city_name"].str.split(":",n=1,expand= True)[0]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [59]:
city_population.head(2)

,year,city_name,id_city,city_population,state
0,1977,AK: Anchorage,22002001.0,174500,AK
1,1978,AK: Anchorage,22002001.0,177000,AK


In [60]:
len(city_population)

6191

In [61]:
city_population["city"]= city_population["city_name"].str.split(":",n=1,expand= True)[1]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
city_population.head(2)

,year,city_name,id_city,city_population,state,city
0,1977,AK: Anchorage,22002001.0,174500,AK,Anchorage
1,1978,AK: Anchorage,22002001.0,177000,AK,Anchorage


- Need to remove all the whitespace in data

In [63]:
city_population['city']=city_population.city.str.lstrip()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
city_population.head(2)

,year,city_name,id_city,city_population,state,city
0,1977,AK: Anchorage,22002001.0,174500,AK,Anchorage
1,1978,AK: Anchorage,22002001.0,177000,AK,Anchorage


In [65]:
len(city_population)

6191

In [66]:
city_population = city_population.drop(columns=['city_name','id_city'])

In [67]:
city_population.head(2)

,year,city_population,state,city
0,1977,174500,AK,Anchorage
1,1978,177000,AK,Anchorage


- I want to use year and combine it with other columns to create a unique primary key. First I need to change the datatype of the "year" column before I combine it with others 


In [68]:
city_population['_year'] = city_population['year'].astype(str)

- Now I can combine year, city and state to form a unique value for each row

In [69]:
city_population["city_population_key"]=city_population['_year']+city_population['city']+city_population['state']

In [70]:
city_population.city_population_key.nunique()

6150

- This is weird because the 3 columns I used to create a unique key shouldn't create any duplicated values. It seems that 41 values have been dupplicated. I will keep this in mind but keep exploring

In [71]:
city_population.head(2)

,year,city_population,state,city,_year,city_population_key
0,1977,174500,AK,Anchorage,1977,1977AnchorageAK
1,1978,177000,AK,Anchorage,1978,1978AnchorageAK


In [72]:
city_population = city_population.drop(columns='_year')
# Dropping unused column

In [73]:
city_population.head(2)

,year,city_population,state,city,city_population_key
0,1977,174500,AK,Anchorage,1977AnchorageAK
1,1978,177000,AK,Anchorage,1978AnchorageAK


In [74]:
city_population = city_population.reindex(columns=['city_population_key',"year","city_population","state","city"])

In [75]:
len(city_population)

6191

In [76]:
city_population.groupby('state').nunique()

,city_population_key,year,city_population,state,city
state,,,,,
AK,82,41,82,1,2
AL,123,41,123,1,3
AR,82,41,82,1,2
AZ,123,41,123,1,3
CA,656,41,656,1,16
CO,123,41,122,1,3
CT,123,41,123,1,3
DC,41,41,41,1,1
DE,82,41,79,1,2


- I noticed that this dataset has median values for cities, which I don't want. So I am going to delete these records. This explains why I had 41 duplicated values for my primary key.

In [77]:
city_population.query('state == "Median for Cities"').count()

city_population_key     0
year                   41
city_population        41
state                  41
city                    0
dtype: int64

- After deleting the 41 records, I should expect to 6150 records in the dataset

In [78]:
city_population = city_population.query('state != "Median for Cities"')

In [79]:
len(city_population)

6150

In [80]:
city_population_file = 'city_population.csv'

In [225]:
city_population.to_csv(os.path.join(s3path, city_population_file), header=True, index = False)

#### 3. Police Killings

In [82]:
police_killings_df.head(2)

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,31/12/2019,Broad St.,Globe,AZ,85501.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name withheld by police,NaN,Male,Unknown race,NaN,31/12/2019,7239-7411 I-40,Memphis,AR,38103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
police_killings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7907 entries, 0 to 7906
Data columns (total 66 columns):
 #   Column                                                                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                                                                      --------------  -----  
 0   Victim's name                                                                                                                                                               7663 non-null   object 
 1   Victim's age                                                                                                                                                                7596 non-null   object 
 2   Victim's gender                                                                     

- There are a lot of unmaned columns with no value. I will delete them

In [84]:
len(police_killings_df)

7907

- This dataset has 7907 rows

In [85]:
police_killings_cleaned = police_killings_df[["Victim's name", "Victim's age", "Victim's gender","Victim's race","URL of image of victim","Date of Incident (month/day/year)","Street Address of Incident"\
                                             ,"City","State","Zipcode","County","Agency responsible for death","Cause of death","A brief description of the circumstances surrounding the death"\
                                             ,"Official disposition of death (justified or other)","Criminal Charges?","Link to news article or photo of official document","Symptoms of mental illness?"\
                                             ,"Unarmed","Alleged Weapon (Source: WaPo)","Alleged Threat Level (Source: WaPo)","Fleeing (Source: WaPo)","Body Camera (Source: WaPo)","WaPo ID (If included in WaPo database)"\
                                             ,"Off-Duty Killing?","Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )"\
                                             ,"ID"]]

In [86]:
police_killings_cleaned.count()

Victim's name                                                                                                                                                                 7663
Victim's age                                                                                                                                                                  7596
Victim's gender                                                                                                                                                               7655
Victim's race                                                                                                                                                                 7663
URL of image of victim                                                                                                                                                        4200
Date of Incident (month/day/year)                                                                        

In [87]:
police_killings_cleaned.head(3)

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),ID
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,31/12/2019,Broad St.,Globe,AZ,85501.0,...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,Rural,7664.0
1,Name withheld by police,NaN,Male,Unknown race,NaN,31/12/2019,7239-7411 I-40,Memphis,AR,38103.0,...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,Urban,7665.0
2,Terry Hudson,57,Male,Black,NaN,31/12/2019,3600 N 24th St,Omaha,NE,68110.0,...,No,Allegedly Armed,gun,attack,not fleeing,no,5359.0,NaN,Urban,7661.0


- The names of columns are long and messy, I am going to tidy them up

In [88]:
new_columns = {
   "Victim's name":"victim_name",
   "Victim's age":"victim_age",
   "Victim's race":"victim_race",
   "Victim's gender":"victim_gender",
   "URL of image of victim":"victim_image",
   "Date of Incident (month/day/year)":"date_of_incident",
   "Street Address of Incident":"street_address_of_incident",
   "City":"city",
   "State":"state",
   "Zipcode":"zipcode",
   "County":"county",
   "Agency responsible for death":"agency_responsible_for_death",
   "Cause of death":"cause_of_death",
   "A brief description of the circumstances surrounding the death":"desc_death",
   "Official disposition of death (justified or other)":"official_justice_desposition",
   "Criminal Charges?":"has_criminal_charges",
   "Link to news article or photo of official document":"link_to_document",
   "Symptoms of mental illness?": "has_symtoms_of_mental_ilness",
   "Unarmed":'unarmed',
   "Alleged Weapon (Source: WaPo)":"alleged_weapon",
   "Alleged Threat Level (Source: WaPo)":"alleged_threat_level",
   "Fleeing (Source: WaPo)":"fleeing",
   "Body Camera (Source: WaPo)":"body_camera",
   "WaPo ID (If included in WaPo database)":"wapo_id",
   "Off-Duty Killing?":"is_off_duty_killing",
   "Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )":"geography",
   "ID":"id"
}

In [89]:
police_killings_cleaned = police_killings_cleaned.rename(columns=new_columns)

In [90]:
police_killings_cleaned.head(2)

,victim_name,victim_age,victim_gender,victim_race,victim_image,date_of_incident,street_address_of_incident,city,state,zipcode,...,has_symtoms_of_mental_ilness,unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,wapo_id,is_off_duty_killing,geography,id
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,31/12/2019,Broad St.,Globe,AZ,85501.0,...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,Rural,7664.0
1,Name withheld by police,NaN,Male,Unknown race,NaN,31/12/2019,7239-7411 I-40,Memphis,AR,38103.0,...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,Urban,7665.0


In [91]:
police_killings_cleaned.nunique()

victim_name                     7411
victim_age                        87
victim_gender                      4
victim_race                        8
victim_image                    4191
date_of_incident                2404
street_address_of_incident      7487
city                            2885
state                             51
zipcode                         4969
county                          1112
agency_responsible_for_death    2855
cause_of_death                    30
desc_death                      7581
official_justice_desposition     100
has_criminal_charges              29
link_to_document                7560
has_symtoms_of_mental_ilness       7
unarmed                            4
alleged_weapon                   172
alleged_threat_level               3
fleeing                            8
body_camera                        5
wapo_id                         4876
is_off_duty_killing                1
geography                          3
id                              7663
d

- I noticed that there are some duplicated names and I want to know that those duplicated values are.

In [92]:
police_killings_cleaned.victim_name.value_counts().head(5)

Name withheld by police    210
Robert Edwards               3
Michael Brown                3
Victor Hernandez             2
Eduardo Rodriguez            2
Name: victim_name, dtype: int64

- Now this is clearer. The names are withheld by police. For sake of data integrity, I will not delete these records


In [93]:
police_killings_cleaned.query("victim_name == 'Name withheld by police'").count()

victim_name                     210
victim_age                      154
victim_gender                   205
victim_race                     210
victim_image                      0
date_of_incident                210
street_address_of_incident      198
city                            209
state                           210
zipcode                         204
county                          205
agency_responsible_for_death    205
cause_of_death                  210
desc_death                      205
official_justice_desposition    206
has_criminal_charges            210
link_to_document                205
has_symtoms_of_mental_ilness    207
unarmed                         210
alleged_weapon                  210
alleged_threat_level            147
fleeing                         132
body_camera                     117
wapo_id                         120
is_off_duty_killing               9
geography                       204
id                              210
dtype: int64

In [94]:
police_killings_cleaned.state.unique()

array(['AZ', 'AR', 'NE', 'WA', 'MO', 'WV', 'MS', 'KS', 'TX', 'CA', 'SD',
       'IN', 'AK', 'OH', 'TN', 'MD', 'VA', 'UT', 'ME', 'FL', 'GA', 'OK',
       'PA', 'NY', 'NC', 'CO', 'MT', 'HI', 'WI', 'MN', 'MI', 'AL', 'SC',
       'NJ', 'IA', 'LA', 'OR', 'DE', 'NV', 'IL', 'ID', 'MA', 'KY', 'NM',
       'VT', 'WY', 'DC', 'CT', 'NH', 'RI', 'ND', nan], dtype=object)

In [95]:
police_killings_cleaned.state.nunique()

51

In [96]:
police_killings_cleaned[(police_killings_cleaned.state.isnull())].head(20)

,victim_name,victim_age,victim_gender,victim_race,victim_image,date_of_incident,street_address_of_incident,city,state,zipcode,...,has_symtoms_of_mental_ilness,unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,wapo_id,is_off_duty_killing,geography,id
7663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Seems like all the records with no state also do not have any records in other columns. I will delete them too

In [97]:
len(police_killings_cleaned[(police_killings_cleaned.state.isnull())])

244

- 244 records need to be deleted 

In [98]:
police_killings_cleaned = police_killings_cleaned[(police_killings_cleaned.state.notnull())]

In [99]:
len(police_killings_cleaned)

7663

- Original dataset had 7907 reccords, minus 244 and we get 7663 records

#### Now I will look at the values of columns and see if I can group up some information

In [100]:
police_killings_cleaned.cause_of_death.unique()

array(['Gunshot', 'Gunshot, Taser', 'Physical Restraint', 'Taser',
       'Beaten/Bludgeoned with instrument', 'Beaten', 'Vehicle',
       'Asphyxiated', 'Other', 'Gunshot, Beanbag Gun',
       'Gunshot, Police Dog', 'Gunshot, Bean Bag Gun',
       'Gunshot, Pepper Spray', 'Bean bag', 'Gunshot, Taser, Baton',
       'Gunshot, Taser, Beanbag Shotgun', 'Pepper Spray',
       'Gunshot, Unspecified Less Lethal Weapon', 'Physical restraint',
       'Taser, Baton', 'Bomb', 'Taser, Physical Restraint',
       'Baton, Pepper Spray, Physical Restraint', 'Gunshot, Vehicle',
       'Gunshot, Stabbed', 'Gunshot, Taser, Pepper spray',
       'Taser, Pepper Spray, Beaten', 'Taser, Beaten',
       'Taser, Pepper spray, beaten', 'Tasered'], dtype=object)

In [101]:
police_killings_cleaned.cause_of_death.value_counts()

Gunshot                                    7059
Taser                                       246
Gunshot, Taser                              223
Vehicle                                      33
Beaten                                       30
Asphyxiated                                  14
Physical Restraint                           11
Physical restraint                            9
Other                                         5
Gunshot, Police Dog                           5
Gunshot, Pepper Spray                         4
Pepper Spray                                  4
Bean bag                                      2
Taser, Physical Restraint                     2
Baton, Pepper Spray, Physical Restraint       1
Gunshot, Beanbag Gun                          1
Taser, Pepper spray, beaten                   1
Taser, Pepper Spray, Beaten                   1
Gunshot, Taser, Pepper spray                  1
Beaten/Bludgeoned with instrument             1
Gunshot, Taser, Baton                   

In [102]:
police_killings_cleaned.cause_of_death = police_killings_cleaned.cause_of_death.replace('Taser, Pepper spray, beaten', 'Taser, Pepper Spray, Beaten')
police_killings_cleaned.cause_of_death = police_killings_cleaned.cause_of_death.replace('Physical restraint', 'Physical Restraint')
police_killings_cleaned.cause_of_death = police_killings_cleaned.cause_of_death.replace('Gunshot, Bean Bag Gun', 'Gunshot, Bean Bag, Gun')
police_killings_cleaned.cause_of_death = police_killings_cleaned.cause_of_death.replace('Gunshot, Beanbag Gun', 'Gunshot, Bean Bag, Gun')
police_killings_cleaned.cause_of_death = police_killings_cleaned.cause_of_death.replace('Taser, Pepper spray, beaten', 'Taser, Pepper Spray, Beaten')
police_killings_cleaned.cause_of_death = police_killings_cleaned.cause_of_death.replace('Tasered', 'Taser')
# Grouping up cases with similar kind

In [103]:
police_killings_cleaned.cause_of_death.value_counts()

Gunshot                                    7059
Taser                                       247
Gunshot, Taser                              223
Vehicle                                      33
Beaten                                       30
Physical Restraint                           20
Asphyxiated                                  14
Gunshot, Police Dog                           5
Other                                         5
Gunshot, Pepper Spray                         4
Pepper Spray                                  4
Taser, Pepper Spray, Beaten                   2
Bean bag                                      2
Taser, Physical Restraint                     2
Gunshot, Bean Bag, Gun                        2
Taser, Beaten                                 1
Baton, Pepper Spray, Physical Restraint       1
Gunshot, Taser, Pepper spray                  1
Beaten/Bludgeoned with instrument             1
Gunshot, Taser, Baton                         1
Gunshot, Stabbed                        

In [104]:
police_killings_cleaned.has_criminal_charges.unique()

array(['No known charges', 'Charged with a crime',
       'Charged with manslaughter', 'No', 'Charged, Convicted', 'NO',
       'Charged, Acquitted', 'Charged, Charges Dropped',
       'Charged, Convicted, Sentenced to 5 years in prison',
       'Charged, Convicted, Sentenced to 40 years in prison',
       'Charged, Convicted, Sentenced to 3 months in jail',
       'Charged, Mistrial',
       'Charged, Convicted, Sentenced to 30 years in prison',
       'Charged, Convicted, Sentenced to 2.5 years in prison',
       'Charged, Mistrial, Plead Guilty to Civil Rights Charges',
       'Charged, Convicted, Sentenced to 4 years',
       'Charged, Convicted, Sentenced to 5 years probation.',
       'Charged, Convicted, Sentenced to 20 years in prison',
       'Charged, Convicted, Sentenced to 16 years in prison',
       'Charged, Convicted, Sentenced to life in prison without parole, plus 16 years',
       'Charged, Convicted, Sentenced to 1 year in prison',
       'Charged, Convicted, Sentenc

In [105]:
police_killings_cleaned.has_criminal_charges.value_counts()

No known charges                                                                 7524
Charged with a crime                                                               49
No                                                                                 37
Charged, Acquitted                                                                 17
Charged, Mistrial                                                                   5
Charged, Convicted                                                                  4
Charged, Convicted, Sentenced to 30 years in prison                                 2
Charged, Convicted, Sentenced to 5 years probation.                                 2
Charged, Charges Tossed                                                             2
Charged, Convicted, Sentenced to life in prison                                     2
Charged, Convicted, Sentenced to 18 months                                          1
Charged, Convicted, Sentenced to 20 years in prison   

In [106]:
police_killings_cleaned.has_criminal_charges = police_killings_cleaned.has_criminal_charges.replace('NO', 'No known charges')
police_killings_cleaned.has_criminal_charges = police_killings_cleaned.has_criminal_charges.replace('No', 'No known charges')
# Grouping all the 'No' cases with "No known charges"

In [107]:
police_killings_cleaned.has_criminal_charges.value_counts()

No known charges                                                                 7562
Charged with a crime                                                               49
Charged, Acquitted                                                                 17
Charged, Mistrial                                                                   5
Charged, Convicted                                                                  4
Charged, Convicted, Sentenced to 30 years in prison                                 2
Charged, Convicted, Sentenced to 5 years probation.                                 2
Charged, Charges Tossed                                                             2
Charged, Convicted, Sentenced to life in prison                                     2
Charged, Convicted, Sentenced to 16 years in prison                                 1
Charged, Convicted, Sentenced to 20 years in prison                                 1
Charged, Convicted, Sentenced to 3 months in jail     

In [108]:
police_killings_cleaned.has_symtoms_of_mental_ilness.value_counts()

No                     4765
Yes                    1467
Unknown                1121
Drug or alcohol use     296
unknown                   1
Unkown                    1
Unknown                   1
Name: has_symtoms_of_mental_ilness, dtype: int64

In [109]:
police_killings_cleaned.has_symtoms_of_mental_ilness = police_killings_cleaned.has_symtoms_of_mental_ilness.replace('Unkown', 'Unknown')
police_killings_cleaned.has_symtoms_of_mental_ilness = police_killings_cleaned.has_symtoms_of_mental_ilness.replace('Unknown ', 'Unknown')
police_killings_cleaned.has_symtoms_of_mental_ilness = police_killings_cleaned.has_symtoms_of_mental_ilness.replace('unknown', 'Unknown')
# Grouping all the unknown cases together

In [110]:
police_killings_cleaned.has_symtoms_of_mental_ilness.value_counts()

No                     4765
Yes                    1467
Unknown                1124
Drug or alcohol use     296
Name: has_symtoms_of_mental_ilness, dtype: int64

In [111]:
police_killings_cleaned.victim_gender.unique()

array(['Male', 'Female', nan, 'Transgender', 'Unknown'], dtype=object)

In [112]:
police_killings_cleaned.victim_gender.value_counts()

Male           7253
Female          391
Transgender       7
Unknown           4
Name: victim_gender, dtype: int64

In [113]:
police_killings_cleaned.unarmed.unique()

array(['Allegedly Armed', 'Unclear', 'Vehicle', 'Unarmed'], dtype=object)

In [114]:
police_killings_cleaned.unarmed.value_counts()

Allegedly Armed    5428
Unarmed            1073
Unclear             649
Vehicle             513
Name: unarmed, dtype: int64

In [115]:
police_killings_cleaned.unarmed.unique()

array(['Allegedly Armed', 'Unclear', 'Vehicle', 'Unarmed'], dtype=object)

In [116]:
police_killings_cleaned.victim_race.unique()

array(['White', 'Unknown race', 'Black', 'Hispanic', 'Pacific Islander',
       'Asian', 'Native American', 'Unknown Race'], dtype=object)

- I will group 'Unknown race' and 'Unknown Race' together

In [117]:
police_killings_cleaned.victim_race = police_killings_cleaned.victim_race.replace('Unknown race', 'Unknown Race')


In [118]:
police_killings_cleaned.victim_race.unique()

array(['White', 'Unknown Race', 'Black', 'Hispanic', 'Pacific Islander',
       'Asian', 'Native American'], dtype=object)

In [119]:
# I want to extract racial information for later use
race_list = police_killings_cleaned.victim_race.unique()

In [120]:
police_killings_cleaned.alleged_threat_level.unique()

array(['other', 'attack', nan, 'undetermined'], dtype=object)

In [121]:
police_killings_cleaned.alleged_threat_level.value_counts()

attack          3395
other           1595
undetermined     291
Name: alleged_threat_level, dtype: int64

In [122]:
police_killings_cleaned.fleeing.unique()

array(['not fleeing', nan, 'car', 'Foot', 'Not fleeing', '0', 'foot',
       'Car', 'Other'], dtype=object)

In [123]:
police_killings_cleaned.fleeing.value_counts()

Not fleeing    3365
Car             808
Foot            632
Other           144
0                83
not fleeing       8
car               5
foot              2
Name: fleeing, dtype: int64

In [124]:
police_killings_cleaned[(police_killings_cleaned.fleeing == '0')].head(5)

,victim_name,victim_age,victim_gender,victim_race,victim_image,date_of_incident,street_address_of_incident,city,state,zipcode,...,has_symtoms_of_mental_ilness,unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,wapo_id,is_off_duty_killing,geography,id
11,John Bott,76,Male,Unknown Race,NaN,29/12/2019,US-78,Byhalia,MS,38611.0,...,Unknown,Allegedly Armed,gun,undetermined,0,No,5310.0,NaN,Rural,7654.0
23,Christopher Camacho,16,Male,Unknown Race,NaN,27/12/2019,138 Washington St,Limerick,ME,4048.0,...,No,Unclear,undetermined,undetermined,0,No,5322.0,NaN,Rural,7647.0
24,Antonio Smith,35,Male,Black,https://fatalencounters.org/wp-content/uploads...,26/12/2019,3600 Hallbrook St,Memphis,TN,38127.0,...,No,Unclear,unknown weapon,other,0,No,5314.0,NaN,Suburban,7643.0
43,Gary Wayne Madewell,38,Male,White,https://fatalencounters.org/wp-content/uploads...,19/12/2019,5 Boyd Ln,Carthage,TN,37030.0,...,No,Allegedly Armed,knife,other,0,No,5294.0,NaN,Rural,7620.0
54,Dana Brown,27,Male,Pacific Islander,https://fatalencounters.org/wp-content/uploads...,17/12/2019,Malakole Street,Kapolei,HI,96707.0,...,No,Allegedly Armed,knife,attack,0,Yes,5290.0,NaN,Suburban,7616.0


- There are some duplicated values due to use of capital letters. I will tidy it up. I will also group '0' under 'Other'

In [125]:
police_killings_cleaned.fleeing = police_killings_cleaned.fleeing.replace('not fleeing', 'Not fleeing')
police_killings_cleaned.fleeing = police_killings_cleaned.fleeing.replace('car', 'Car')
police_killings_cleaned.fleeing = police_killings_cleaned.fleeing.replace('foot', 'Foot')
police_killings_cleaned.fleeing = police_killings_cleaned.fleeing.replace('0', 'Other')

In [126]:
police_killings_cleaned.fleeing.value_counts()

Not fleeing    3373
Car             813
Foot            634
Other           227
Name: fleeing, dtype: int64

In [127]:
police_killings_cleaned.is_off_duty_killing.unique()

array([nan, 'Off-Duty'], dtype=object)

In [128]:
police_killings_cleaned.is_off_duty_killing.nunique()

1

In [129]:
police_killings_cleaned.alleged_weapon.unique()

array(['knife', 'unclear', 'gun', 'vehicle', 'chain', 'unknown weapon',
       'Taser', 'undetermined', 'gun and vehicle', 'shovel',
       'gun and car', 'toy weapon', 'unarmed', 'baseball bat', 'hatchet',
       'car, knife and mace', 'sword', 'vehicle and machete',
       'screwdriver', 'vehicle and gun', 'BB gun', 'sharp object',
       'box cutter', 'ax', 'hammer', 'crowbar', 'chair', 'scissors',
       'toy', 'baseball bat and knife', 'straight edge razor', 'machete',
       'baton', 'Airsoft pistol', 'air pistol', 'wasp spray',
       'BB gun and vehicle', 'piece of wood', 'gun and knife',
       'garden tool', 'barstool', 'walking stick', 'wrench',
       'beer bottle', 'meat cleaver', 'metal pipe', 'flag pole', 'rock',
       'lawn mower blade', 'crossbow', 'metal object', 'bow and arrow',
       'pick-axe', 'lamp', 'glass shard', 'incendiary device', 'unknown',
       'pipe', 'pole and knife', 'Unknown weapon', 'lighter fluid',
       'pitchfork', 'gun and sword', 'taser', 'b

- There are too many items in the weapon column. I am not going to tidy it up

In [130]:
police_killings_cleaned.geography.unique()

array(['Rural', 'Urban', 'Suburban', nan], dtype=object)

In [131]:
police_killings_cleaned.geography.nunique()

3

In [132]:
police_killings_cleaned.nunique()

victim_name                     7411
victim_age                        87
victim_gender                      4
victim_race                        7
victim_image                    4191
date_of_incident                2404
street_address_of_incident      7487
city                            2885
state                             51
zipcode                         4969
county                          1112
agency_responsible_for_death    2855
cause_of_death                    26
desc_death                      7581
official_justice_desposition     100
has_criminal_charges              27
link_to_document                7560
has_symtoms_of_mental_ilness       4
unarmed                            4
alleged_weapon                   172
alleged_threat_level               3
fleeing                            4
body_camera                        5
wapo_id                         4876
is_off_duty_killing                1
geography                          3
id                              7663
d

In [133]:
len(police_killings_cleaned)

7663

In [134]:
police_killings_cleaned = police_killings_cleaned.rename(columns={"id":"victim_id"})
# renaming

In [135]:
police_killings_cleaned.head(2)

,victim_name,victim_age,victim_gender,victim_race,victim_image,date_of_incident,street_address_of_incident,city,state,zipcode,...,has_symtoms_of_mental_ilness,unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,wapo_id,is_off_duty_killing,geography,victim_id
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,31/12/2019,Broad St.,Globe,AZ,85501.0,...,No,Allegedly Armed,knife,other,Not fleeing,no,5332.0,NaN,Rural,7664.0
1,Name withheld by police,NaN,Male,Unknown Race,NaN,31/12/2019,7239-7411 I-40,Memphis,AR,38103.0,...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,Urban,7665.0


- I want to change the datatype of some columns so that they can be joined with other datasets during analysis

In [136]:
police_killings_cleaned.victim_id = police_killings_cleaned.victim_id.astype(int)
police_killings_cleaned.zipcode = police_killings_cleaned.zipcode.astype(str)
police_killings_cleaned.wapo_id = police_killings_cleaned.wapo_id.astype(str)
police_killings_cleaned.date_of_incident = police_killings_cleaned.date_of_incident.astype('datetime64[ns]') 

In [137]:
police_killings_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7663 entries, 0 to 7662
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   victim_name                   7663 non-null   object        
 1   victim_age                    7596 non-null   object        
 2   victim_gender                 7655 non-null   object        
 3   victim_race                   7663 non-null   object        
 4   victim_image                  4200 non-null   object        
 5   date_of_incident              7663 non-null   datetime64[ns]
 6   street_address_of_incident    7580 non-null   object        
 7   city                          7657 non-null   object        
 8   state                         7663 non-null   object        
 9   zipcode                       7663 non-null   object        
 10  county                        7648 non-null   object        
 11  agency_responsible_for_death  

In [138]:
police_killings_cleaned.head(1)

,victim_name,victim_age,victim_gender,victim_race,victim_image,date_of_incident,street_address_of_incident,city,state,zipcode,...,has_symtoms_of_mental_ilness,unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,wapo_id,is_off_duty_killing,geography,victim_id
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,2019-12-31,Broad St.,Globe,AZ,85501.0,...,No,Allegedly Armed,knife,other,Not fleeing,no,5332.0,NaN,Rural,7664


In [139]:
new_order = ["victim_id","victim_name","victim_gender"\
            ,"victim_race","victim_image","date_of_incident"\
            ,"street_address_of_incident","city","state","zipcode"\
            ,"county","agency_responsible_for_death","cause_of_death"\
            ,"desc_death","official_justice_desposition","has_criminal_charges"\
            ,"link_to_document","has_symtoms_of_mental_ilness","unarmed","alleged_weapon"\
            ,"alleged_threat_level","fleeing","body_camera","wapo_id","is_off_duty_killing"\
            ,"geography"]

In [140]:
police_killings_cleaned = police_killings_cleaned.reindex(columns=new_order)

In [141]:
police_killings_cleaned.head(1)

,victim_id,victim_name,victim_gender,victim_race,victim_image,date_of_incident,street_address_of_incident,city,state,zipcode,...,link_to_document,has_symtoms_of_mental_ilness,unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,wapo_id,is_off_duty_killing,geography
0,7664,Eric M. Tellez,Male,White,https://fatalencounters.org/wp-content/uploads...,2019-12-31,Broad St.,Globe,AZ,85501.0,...,https://www.azfamily.com/news/phoenix-man-arme...,No,Allegedly Armed,knife,other,Not fleeing,no,5332.0,NaN,Rural


In [142]:
victims_list = 'victims.csv'

In [224]:
police_killings_cleaned.to_csv(os.path.join(s3path, victims_list),header=True, index = False)

#### 4. Police Deaths

In [144]:
police_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   person       22800 non-null  object
 1   dept         22800 non-null  object
 2   eow          22800 non-null  object
 3   cause        22800 non-null  object
 4   cause_short  22800 non-null  object
 5   date         22800 non-null  object
 6   year         22800 non-null  int64 
 7   canine       22800 non-null  bool  
 8   dept_name    22800 non-null  object
 9   state        22800 non-null  object
dtypes: bool(1), int64(1), object(8)
memory usage: 1.6+ MB


In [145]:
police_deaths_df_cleaned = police_deaths_df

In [146]:
police_deaths_df_cleaned.nunique()

person         22742
dept            6528
eow            17158
cause             36
cause_short       36
date           17158
year             202
canine             2
dept_name       5525
state             60
dtype: int64

In [147]:
police_deaths_df_cleaned.head()

,person,dept,eow,cause,cause_short,date,year,canine,dept_name,state
0,Constable Darius Quimby,"Albany County Constable's Office, NY","EOW: Monday, January 3, 1791",Cause of Death: Gunfire,Gunfire,1791-01-03,1791,False,Albany County Constable's Office,NY
1,Sheriff Cornelius Hogeboom,"Columbia County Sheriff's Office, NY","EOW: Saturday, October 22, 1791",Cause of Death: Gunfire,Gunfire,1791-10-22,1791,False,Columbia County Sheriff's Office,NY
2,Deputy Sheriff Isaac Smith,"Westchester County Sheriff's Department, NY","EOW: Thursday, May 17, 1792",Cause of Death: Gunfire,Gunfire,1792-05-17,1792,False,Westchester County Sheriff's Department,NY
3,Marshal Robert Forsyth,United States Department of Justice - United S...,"EOW: Saturday, January 11, 1794",Cause of Death: Gunfire,Gunfire,1794-01-11,1794,False,United States Department of Justice - United S...,US
4,Sheriff Robert Maxwell,"Greenville County Sheriff's Office, SC","EOW: Sunday, November 12, 1797",Cause of Death: Gunfire,Gunfire,1797-11-12,1797,False,Greenville County Sheriff's Office,SC


In [148]:
police_deaths_df_cleaned.cause.value_counts()


Cause of Death: Gunfire                     12067
Cause of Death: Automobile accident          2348
Cause of Death: Motorcycle accident          1134
Cause of Death: Heart attack                  977
Cause of Death: Vehicular assault             888
Cause of Death: Struck by vehicle             868
Cause of Death: Vehicle pursuit               627
Cause of Death: Assault                       613
Cause of Death: Gunfire (Accidental)          604
Cause of Death: Stabbed                       465
Cause of Death: Aircraft accident             381
Cause of Death: Drowned                       262
Cause of Death: Struck by train               254
Cause of Death: Fall                          197
Cause of Death: Duty related illness          169
Cause of Death: 9/11 related illness          110
Cause of Death: Terrorist attack               85
Cause of Death: Electrocuted                   83
Cause of Death: Animal related                 82
Cause of Death: Accidental                     80


In [149]:
police_deaths_df_cleaned.cause.nunique()

36

In [150]:
police_deaths_df.cause_short.value_counts()

Gunfire                     12067
Automobile accident          2348
Motorcycle accident          1134
Heart attack                  977
Vehicular assault             888
Struck by vehicle             868
Vehicle pursuit               627
Assault                       613
Gunfire (Accidental)          604
Stabbed                       465
Aircraft accident             381
Drowned                       262
Struck by train               254
Fall                          197
Duty related illness          169
9/11 related illness          110
Terrorist attack               85
Electrocuted                   83
Animal related                 82
Accidental                     80
Struck by streetcar            68
Fire                           63
Heat exhaustion                61
Explosion                      58
Bomb                           57
Exposure to toxins             44
Weather/Natural disaster       33
Training accident              28
Structure collapse             23
Train accident

In [151]:
police_deaths_df_cleaned.cause_short.nunique()

36

- Column 'cause' and 'cause_short' are the same. I will drop 'cause column'

In [152]:
police_deaths_df_cleaned.cause_short.value_counts()

Gunfire                     12067
Automobile accident          2348
Motorcycle accident          1134
Heart attack                  977
Vehicular assault             888
Struck by vehicle             868
Vehicle pursuit               627
Assault                       613
Gunfire (Accidental)          604
Stabbed                       465
Aircraft accident             381
Drowned                       262
Struck by train               254
Fall                          197
Duty related illness          169
9/11 related illness          110
Terrorist attack               85
Electrocuted                   83
Animal related                 82
Accidental                     80
Struck by streetcar            68
Fire                           63
Heat exhaustion                61
Explosion                      58
Bomb                           57
Exposure to toxins             44
Weather/Natural disaster       33
Training accident              28
Structure collapse             23
Train accident

In [153]:
police_deaths_df_cleaned.state.value_counts()

 TX    1880
 CA    1639
 NY    1574
 US    1225
 IL    1073
 PA     901
 OH     834
 FL     820
 KY     772
 GA     698
 MO     666
 TN     607
 MI     574
 NC     549
 AL     537
 VA     523
 OK     496
 NJ     485
 LA     470
 IN     434
 PR     363
 SC     362
 MA     360
 MD     325
 CO     305
 WA     290
 AR     284
 RR     280
 WI     275
 AZ     273
 MS     261
 KS     257
 MN     244
 WV     208
 IA     197
 OR     182
 NM     151
 CT     144
 UT     128
 DC     128
 MT     128
 NV     120
 NE     110
 ME      86
 ID      64
 WY      57
 HI      56
 ND      55
 SD      53
 TR      51
 RI      50
 AK      48
 NH      46
 DE      41
 VT      26
 VI      14
 GU      12
 MP       3
 AS       3
 CZ       3
Name: state, dtype: int64

In [154]:
police_deaths_df_cleaned.query('state == " US"')
# I need to get rid of the whitespace

,person,dept,eow,cause,cause_short,date,year,canine,dept_name,state
3,Marshal Robert Forsyth,United States Department of Justice - United S...,"EOW: Saturday, January 11, 1794",Cause of Death: Gunfire,Gunfire,1794-01-11,1794,False,United States Department of Justice - United S...,US
10,Boatman Asa Marsh,United States Department of the Treasury - Cus...,"EOW: Wednesday, August 3, 1808",Cause of Death: Gunfire,Gunfire,1808-08-03,1808,False,United States Department of the Treasury - Cus...,US
11,Boatman Ellis Drake,United States Department of the Treasury - Cus...,"EOW: Wednesday, August 3, 1808",Cause of Death: Gunfire,Gunfire,1808-08-03,1808,False,United States Department of the Treasury - Cus...,US
12,Custom House Officer Daniel Miller,United States Department of the Treasury - Cus...,"EOW: Monday, January 1, 1810",Cause of Death: Gunfire,Gunfire,1810-01-01,1810,False,United States Department of the Treasury - Cus...,US
14,Customs Inspector John Stout,United States Department of the Treasury - Cus...,"EOW: Friday, January 21, 1814",Cause of Death: Gunfire,Gunfire,1814-01-21,1814,False,United States Department of the Treasury - Cus...,US
...,...,...,...,...,...,...,...,...,...,...
22728,Special Agent Scott McGuire,United States Department of Homeland Security ...,"EOW: Sunday, January 24, 2016",Cause of Death: Vehicular assault,Vehicular assault,2016-01-24,2016,False,United States Department of Homeland Security ...,US
22756,Border Patrol Agent Jose Daniel Barraza,United States Department of Homeland Security ...,"EOW: Monday, April 18, 2016",Cause of Death: Automobile accident,Automobile accident,2016-04-18,2016,False,United States Department of Homeland Security ...,US
22767,Deportation Officer Brian Beliso,United States Department of Homeland Security ...,"EOW: Wednesday, June 8, 2016",Cause of Death: Heart attack,Heart attack,2016-06-08,2016,False,United States Department of Homeland Security ...,US
22774,Officer Bradley Wayne Treat,United States Department of Agriculture - Fore...,"EOW: Wednesday, June 29, 2016",Cause of Death: Animal related,Animal related,2016-06-29,2016,False,United States Department of Agriculture - Fore...,US


In [155]:
police_deaths_df_cleaned['state']=police_deaths_df['state'].str.strip()

In [156]:
police_deaths_df_cleaned.query('state == "US"').count()

person         1225
dept           1225
eow            1225
cause          1225
cause_short    1225
date           1225
year           1225
canine         1225
dept_name      1225
state          1225
dtype: int64

- Although the US is not a state, I don't want to remove all the records 

In [157]:
police_deaths_df_cleaned.dept.value_counts()

New York City Police Department, NY                                         753
Chicago Police Department, IL                                               492
Puerto Rico Police Department, PR                                           338
United States Department of Justice - United States Marshals Service, US    279
Philadelphia Police Department, PA                                          256
                                                                           ... 
Allendale County Sheriff's Department, SC                                     1
Kilgore Police Department, TX                                                 1
Polk County Sheriff's Office, GA                                              1
Goochland County Sheriff's Office, VA                                         1
Avilla Police Department, IN                                                  1
Name: dept, Length: 6528, dtype: int64

In [158]:
police_deaths_df_cleaned.dept_name.value_counts()

New York City Police Department                                         753
Chicago Police Department                                               492
Puerto Rico Police Department                                           338
United States Department of Justice - United States Marshals Service    279
Philadelphia Police Department                                          256
                                                                       ... 
Galena Park Police Department                                             1
McLean County Coroner's Office                                            1
Cayce Police Department                                                   1
Ivanhoe Police Department                                                 1
Matewan Police Department                                                 1
Name: dept_name, Length: 5525, dtype: int64

- Dept and dept_name share the same information. Given that we already have the State column, I will drop 'dept' column

In [159]:
police_deaths_df_cleaned.date = police_deaths_df_cleaned.date.astype('datetime64[ns]') 

In [160]:
police_deaths_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   person       22800 non-null  object        
 1   dept         22800 non-null  object        
 2   eow          22800 non-null  object        
 3   cause        22800 non-null  object        
 4   cause_short  22800 non-null  object        
 5   date         22800 non-null  datetime64[ns]
 6   year         22800 non-null  int64         
 7   canine       22800 non-null  bool          
 8   dept_name    22800 non-null  object        
 9   state        22800 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(7)
memory usage: 1.6+ MB


- Given eow and date are the same, I will drop eow column


In [161]:
police_deaths_df_cleaned = police_deaths_df_cleaned.drop(columns=['eow','cause','dept'])

In [162]:
police_deaths_df_cleaned = police_deaths_df_cleaned.rename(columns={"cause_short": "cause_of_death"})

In [163]:
police_deaths_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   person          22800 non-null  object        
 1   cause_of_death  22800 non-null  object        
 2   date            22800 non-null  datetime64[ns]
 3   year            22800 non-null  int64         
 4   canine          22800 non-null  bool          
 5   dept_name       22800 non-null  object        
 6   state           22800 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(4)
memory usage: 1.1+ MB


In [164]:
len(police_deaths_df_cleaned)

22800

In [165]:
police_deaths_df_cleaned.person.value_counts()

K9 Rocky                                   4
K9 Bandit                                  3
K9 Sarge                                   3
K9 King                                    3
K9 Bruno                                   3
                                          ..
Customs Officer Clyde M. Bristow           1
Patrolman Jeff Evans                       1
Patrolman Gordon W. Swettart               1
Officer Nelson S. Dwelly                   1
Corrections Employee Harold Edward Rice    1
Name: person, Length: 22742, dtype: int64

- Some names are duplicated but they seem to refer to a group of police. Given that I do not have more information about these records, I will keep them there

In [166]:
police_death_file = 'police_death.csv'

In [223]:
police_deaths_df_cleaned.to_csv(os.path.join(s3path, police_death_file), header=True, index = False)

#### 5. Protest

In [168]:
protests_df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- geometry: struct (nullable = true)
 |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- properties: struct (nullable = true)
 |    |    |    |-- Alias: string (nullable = true)
 |    |    |    |-- Batch: string (nullable = true)
 |    |    |    |-- City: string (nullable = true)
 |    |    |    |-- Comments: string (nullable = true)
 |    |    |    |-- Country: string (nullable = true)
 |    |    |    |-- Date_Added: long (nullable = true)
 |    |    |    |-- Mailing_Abbreviation: string (nullable = true)
 |    |    |    |-- OBJECTID: long (nullable = true)
 |    |    |    |-- Region: string (nullable = true)
 |    |    |    |-- Search_Label: string (nullable = true)
 |    |    |    |-- X_Longitude: double (n

- I am going to use spark explode function to flatten the data

In [169]:
from pyspark.sql.functions import explode


In [170]:
protests_df_exploded = protests_df.select('type',explode('features').alias('features'))

In [171]:
protests_cleaned = protests_df_exploded.select(col('type').alias('featuretype'),'features.geometry.type','features.id','features.properties.Alias'\
                    ,'features.properties.Batch','features.properties.City','features.properties.Comments','features.properties.Country'\
                   ,'features.properties.Date_Added','features.properties.Mailing_Abbreviation','features.properties.OBJECTID'\
                   ,'features.properties.Region','features.properties.Search_Label','features.properties.X_Longitude','features.properties.Y_Latitude')

In [172]:
protests_cleaned.show(20)

+-----------------+-----+---+-----+-----------+------------+--------+-------+-------------+--------------------+--------+--------+--------------------+-------------------+------------------+
|      featuretype| type| id|Alias|      Batch|        City|Comments|Country|   Date_Added|Mailing_Abbreviation|OBJECTID|  Region|        Search_Label|        X_Longitude|        Y_Latitude|
+-----------------+-----+---+-----+-----------+------------+--------+-------+-------------+--------------------+--------+--------+--------------------+-------------------+------------------+
|FeatureCollection|Point|  1| null|2020_0602_1|      Juneau|    null|    USA|1591056000000|                  AK|       1|  Alaska| Juneau, Alaska, USA|-134.40678999999994| 58.29973000000007|
|FeatureCollection|Point|  2| null|2020_0602_1|   Anchorage|    null|    USA|1591056000000|                  AK|       2|  Alaska|Anchorage, Alaska...|-149.85824999999997| 61.21753000000007|
|FeatureCollection|Point|  3| null|2020_0602_

In [173]:
protests_cleaned.printSchema()

root
 |-- featuretype: string (nullable = true)
 |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- Alias: string (nullable = true)
 |-- Batch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Comments: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Date_Added: long (nullable = true)
 |-- Mailing_Abbreviation: string (nullable = true)
 |-- OBJECTID: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- Search_Label: string (nullable = true)
 |-- X_Longitude: double (nullable = true)
 |-- Y_Latitude: double (nullable = true)



In [174]:
# Some columns seem to be showing single value and they are not that relevant. I will delete them if that's the case
protests_cleaned.groupBy('featuretype').count().show(100)

+-----------------+-----+
|      featuretype|count|
+-----------------+-----+
|FeatureCollection| 4071|
+-----------------+-----+



In [175]:
protests_cleaned.groupBy('type').count().show(100)

+-----+-----+
| type|count|
+-----+-----+
|Point| 4071|
+-----+-----+



In [176]:
protests_cleaned.groupBy('Alias').count().show(100)

+-------------+-----+
|        Alias|count|
+-------------+-----+
|       Queens|    4|
|            x|    1|
|  Los Angeles|    2|
|    San Diego|    1|
|         null| 4052|
|       London|    1|
|       Barrow|    1|
|            d|    1|
|    Las Vegas|    1|
|Staten Island|    3|
|        Dover|    1|
|      Hagatna|    1|
|New York City|    1|
|             |    1|
+-------------+-----+



- Most are nulls in this column so I will delete this column

In [177]:
protests_cleaned.groupBy('Comments').count().show(100)

+--------------------+-----+
|            Comments|count|
+--------------------+-----+
|                   x|    1|
|                null| 4069|
|Tagline says "Geo...|    1|
+--------------------+-----+



- Most are nulls in this column so I will delete this column

In [178]:
protests_cleaned.groupBy('id','OBJECTID').count().show(30)

+----+--------+-----+
|  id|OBJECTID|count|
+----+--------+-----+
| 346|     346|    1|
| 435|     435|    1|
| 753|     753|    1|
| 787|     787|    1|
|1167|    1167|    1|
|1395|    1395|    1|
|1508|    1508|    1|
|1909|    1909|    1|
|2005|    2005|    1|
|2226|    2226|    1|
|2382|    2382|    1|
|2941|    2941|    1|
|3416|    3416|    1|
|  24|      24|    1|
| 120|     120|    1|
| 707|     707|    1|
| 734|     734|    1|
|1043|    1043|    1|
|1111|    1111|    1|
|1283|    1283|    1|
|2277|    2277|    1|
|2609|    2609|    1|
|2841|    2841|    1|
|3060|    3060|    1|
|3225|    3225|    1|
|3515|    3515|    1|
|3696|    3696|    1|
|3952|    3952|    1|
|3963|    3963|    1|
|4000|    4000|    1|
+----+--------+-----+
only showing top 30 rows



- These two columns are identical. I will delete the 'objectid' column.

In [179]:
protests_cleaned.groupBy('Batch').count().show(10)

+-----------+-----+
|      Batch|count|
+-----------+-----+
|2020_0626_1|    3|
|2020_0604_1|  208|
|2020_0602_1|  517|
|2020_0610_1|   45|
|2020_0609_3|   77|
|2020_0623_1|   18|
|2020_0628_1|   12|
|2020_0618_1|  134|
|2020_0606_2|   99|
|2020_0625_1|    4|
+-----------+-----+
only showing top 10 rows



- This column has various values so I will keep it

In [180]:
columns_to_drop = ['OBJECTID', 'Comments','Alias','type','featuretype']


In [181]:
protests_cleaned = protests_cleaned.drop(*columns_to_drop)

In [182]:
protests_cleaned.show(3)

+---+-----------+---------+-------+-------------+--------------------+------+--------------------+-------------------+-----------------+
| id|      Batch|     City|Country|   Date_Added|Mailing_Abbreviation|Region|        Search_Label|        X_Longitude|       Y_Latitude|
+---+-----------+---------+-------+-------------+--------------------+------+--------------------+-------------------+-----------------+
|  1|2020_0602_1|   Juneau|    USA|1591056000000|                  AK|Alaska| Juneau, Alaska, USA|-134.40678999999994|58.29973000000007|
|  2|2020_0602_1|Anchorage|    USA|1591056000000|                  AK|Alaska|Anchorage, Alaska...|-149.85824999999997|61.21753000000007|
|  3|2020_0602_1|Fairbanks|    USA|1591056000000|                  AK|Alaska|Fairbanks, Alaska...|-147.72209999999998|64.84525000000008|
+---+-----------+---------+-------+-------------+--------------------+------+--------------------+-------------------+-----------------+
only showing top 3 rows



- Now I need to change Date_Added from LongType (epoch) to DateType

In [183]:
from pyspark.sql.types import *
import pyspark.sql.functions as f
protests_cleaned= protests_cleaned.withColumn("date_added", protests_cleaned["Date_Added"])

In [184]:
get_timestamp = udf(lambda x: datetime.utcfromtimestamp(int(x)/1000), DateType())
protests_cleaned = protests_cleaned.withColumn('date_added',get_timestamp('date_added'))

In [185]:
protests_cleaned.show(2)

+---+-----------+---------+-------+----------+--------------------+------+--------------------+-------------------+-----------------+
| id|      Batch|     City|Country|date_added|Mailing_Abbreviation|Region|        Search_Label|        X_Longitude|       Y_Latitude|
+---+-----------+---------+-------+----------+--------------------+------+--------------------+-------------------+-----------------+
|  1|2020_0602_1|   Juneau|    USA|2020-06-02|                  AK|Alaska| Juneau, Alaska, USA|-134.40678999999994|58.29973000000007|
|  2|2020_0602_1|Anchorage|    USA|2020-06-02|                  AK|Alaska|Anchorage, Alaska...|-149.85824999999997|61.21753000000007|
+---+-----------+---------+-------+----------+--------------------+------+--------------------+-------------------+-----------------+
only showing top 2 rows



In [186]:
protests_cleaned.printSchema()

root
 |-- id: long (nullable = true)
 |-- Batch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- Mailing_Abbreviation: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Search_Label: string (nullable = true)
 |-- X_Longitude: double (nullable = true)
 |-- Y_Latitude: double (nullable = true)



In [187]:
protests_cleaned.show()

+---+-----------+------------+-------+----------+--------------------+--------+--------------------+-------------------+------------------+
| id|      Batch|        City|Country|date_added|Mailing_Abbreviation|  Region|        Search_Label|        X_Longitude|        Y_Latitude|
+---+-----------+------------+-------+----------+--------------------+--------+--------------------+-------------------+------------------+
|  1|2020_0602_1|      Juneau|    USA|2020-06-02|                  AK|  Alaska| Juneau, Alaska, USA|-134.40678999999994| 58.29973000000007|
|  2|2020_0602_1|   Anchorage|    USA|2020-06-02|                  AK|  Alaska|Anchorage, Alaska...|-149.85824999999997| 61.21753000000007|
|  3|2020_0602_1|   Fairbanks|    USA|2020-06-02|                  AK|  Alaska|Fairbanks, Alaska...|-147.72209999999998| 64.84525000000008|
|  4|2020_0602_1|     Decatur|    USA|2020-06-02|                  AL| Alabama|Decatur, Alabama,...| -86.97978999999998| 34.60740000000004|
|  5|2020_0602_1|   

In [188]:
protests_cleaned.groupBy('Country').count().show(100)

+-------------------+-----+
|            Country|count|
+-------------------+-----+
|             Russia|    1|
|            Senegal|    1|
|             Sweden|    5|
|        Philippines|    1|
|               Fiji|    1|
|             Turkey|    2|
|            Germany|   49|
|             France|   20|
|             Greece|    2|
|             Kosovo|    1|
|          Sri Lanka|    1|
|             Taiwan|    1|
|           Slovakia|    1|
|          Argentina|    1|
|            Belgium|   10|
|            Ecuador|    1|
|            Finland|    7|
|              Ghana|    1|
|              India|    1|
|              China|    1|
|              Malta|    1|
|            Croatia|    1|
|            Nigeria|    2|
|            Andorra|    1|
|              Italy|   26|
|          Lithuania|    1|
|             Norway|    5|
|              Spain|   22|
|         Guadeloupe|    1|
|            Denmark|    5|
|           Barbados|    1|
|               Iran|    2|
|            Ireland

- This dataset contains a lot of non-US records. Since we are only analysing US data for this project, I will drop all the non-US records

In [189]:
protests_cleaned_df = protests_cleaned.filter(protests_cleaned.Country == "USA")

In [190]:
protests_cleaned_df.show(2)

+---+-----------+---------+-------+----------+--------------------+------+--------------------+-------------------+-----------------+
| id|      Batch|     City|Country|date_added|Mailing_Abbreviation|Region|        Search_Label|        X_Longitude|       Y_Latitude|
+---+-----------+---------+-------+----------+--------------------+------+--------------------+-------------------+-----------------+
|  1|2020_0602_1|   Juneau|    USA|2020-06-02|                  AK|Alaska| Juneau, Alaska, USA|-134.40678999999994|58.29973000000007|
|  2|2020_0602_1|Anchorage|    USA|2020-06-02|                  AK|Alaska|Anchorage, Alaska...|-149.85824999999997|61.21753000000007|
+---+-----------+---------+-------+----------+--------------------+------+--------------------+-------------------+-----------------+
only showing top 2 rows



In [191]:
protests_cleaned_df.groupBy('Mailing_Abbreviation','Region').count().show(100)

+--------------------+--------------------+-----+
|Mailing_Abbreviation|              Region|count|
+--------------------+--------------------+-----+
|                  MT|             Montana|   17|
|                  NC|      North Carolina|  101|
|                  MD|            Maryland|   70|
|                  CO|            Colorado|   52|
|                  CT|         Connecticut|   91|
|                  IL|            Illinois|  160|
|                  WY|             Wyoming|   14|
|                  NJ|          New Jersey|  152|
|                  DE|            Delaware|    9|
|                  DC|District of Columbia|    1|
|                  AR|            Arkansas|   29|
|                  LA|           Louisiana|   35|
|                  TN|           Tennessee|   40|
|                  AK|              Alaska|   25|
|                  CA|          California|  345|
|                  NM|          New Mexico|   24|
|                  UT|                Utah|   15|


- Three records is null and they should be under PA state

In [192]:
protests_cleaned_df.where(F.col("Mailing_Abbreviation").isNull()).show()

+----+-----------+------------+-------+----------+--------------------+------------+--------------------+------------------+------------------+
|  id|      Batch|        City|Country|date_added|Mailing_Abbreviation|      Region|        Search_Label|       X_Longitude|        Y_Latitude|
+----+-----------+------------+-------+----------+--------------------+------------+--------------------+------------------+------------------+
|3969|2020_0627_1|East Liberty|    USA|2020-06-27|                null|Pennsylvania|East Liberty, Pen...|-79.92311331999997| 40.46050443900003|
|3970|2020_0627_1|   Scottdale|    USA|2020-06-27|                null|Pennsylvania|Scottdale, Pennsy...|-79.59483101399996|40.102712506000046|
|3971|2020_0627_1|    Franklin|    USA|2020-06-27|                null|Pennsylvania|Franklin, Pennsyl...|-79.83155752999994| 41.39753612800007|
+----+-----------+------------+-------+----------+--------------------+------------+--------------------+------------------+------------

In [193]:
protests_cleaned_df = protests_cleaned_df.na.fill("PA",subset=["Mailing_Abbreviation"])

In [194]:
protests_cleaned_df.where(F.col("Mailing_Abbreviation").isNull()).show()

+---+-----+----+-------+----------+--------------------+------+------------+-----------+----------+
| id|Batch|City|Country|date_added|Mailing_Abbreviation|Region|Search_Label|X_Longitude|Y_Latitude|
+---+-----+----+-------+----------+--------------------+------+------------+-----------+----------+
+---+-----+----+-------+----------+--------------------+------+------------+-----------+----------+



- Now I am confident that there are no more null values in the mailing_abbreviation column.

In [195]:
protests_cleaned_df = protests_cleaned_df.withColumnRenamed("id","protest_id")\
.withColumnRenamed("Batch","batch")\
.withColumnRenamed("City","city")\
.withColumnRenamed("Country","country")\
.withColumnRenamed("Mailing_Abbreviation","state")\
.withColumnRenamed("Region","region")\
.withColumnRenamed("Search_Label","search_label")\
.withColumnRenamed("X_Longitude","x_longitude")\
.withColumnRenamed("Y_Latitude","y_latitude")

In [196]:
protests_cleaned_df.printSchema()

root
 |-- protest_id: long (nullable = true)
 |-- batch: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- state: string (nullable = false)
 |-- region: string (nullable = true)
 |-- search_label: string (nullable = true)
 |-- x_longitude: double (nullable = true)
 |-- y_latitude: double (nullable = true)



In [227]:
protests_cleaned_df.count()

3399

In [197]:
protest_file = 'protest.csv'

In [222]:
protests_cleaned_df.toPandas().to_csv(os.path.join(s3path, protest_file),header=True)

#### 6. NY Crimes

In [199]:
ny_df.show(3)

+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|          Latitude|         Longitude|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
| 190294601| 11/23/2018|  109|ASSAULT 2,1,UNCLA...|  106|      FELONY ASSAULT|PL 1200512|         F|          B|             40|                0|    25-44|       M|         BLACK| 1008096.0|  233595.0|40.807816227

In [200]:
ny_df.count()

5012956

In [201]:
ny_df.printSchema()

root
 |-- ARREST_KEY: integer (nullable = true)
 |-- ARREST_DATE: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: integer (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- X_COORD_CD: double (nullable = true)
 |-- Y_COORD_CD: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lon_Lat: string (nullable = true)



In [202]:
ny_df_cleaned = ny_df

- I want to change the arrest_date from stringtype to datetype

In [203]:
#This udf function would convert all the string cells into DateType
get_date =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

ny_df_cleaned = ny_df_cleaned.withColumn('arrest_date', get_date(col('ARREST_DATE')))


In [204]:
ny_df_cleaned.show(2)

+----------+-----------+-----+--------------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
|ARREST_KEY|arrest_date|PD_CD|             PD_DESC|KY_CD|     OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|          Latitude|         Longitude|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+------------------+--------------------+
| 190294601| 2018-11-23|  109|ASSAULT 2,1,UNCLA...|  106|FELONY ASSAULT|PL 1200512|         F|          B|             40|                0|    25-44|       M|         BLACK| 1008096.0|  233595.0|40.807816227000046|-73.9138626609999

In [205]:
ny_df_cleaned.printSchema()

root
 |-- ARREST_KEY: integer (nullable = true)
 |-- arrest_date: date (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: integer (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- X_COORD_CD: double (nullable = true)
 |-- Y_COORD_CD: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lon_Lat: string (nullable = true)



In [206]:
ny_df_cleaned.groupBy('AGE_GROUP').count().show()


+---------+------+
|AGE_GROUP| count|
+---------+------+
|      944|     2|
|      926|     2|
|     1953|     1|
|      740|     1|
|      940|     3|
|      317|     1|
|      200|     1|
|      938|     3|
|      323|     1|
|      309|     1|
|      446|     1|
|      924|     4|
|      910|     1|
|      895|    13|
|     1918|     1|
|      352|     1|
|      894|     7|
|      951|     3|
|      <18|412285|
|      808|     1|
+---------+------+
only showing top 20 rows



- data is clean so no need to fix 

In [207]:
ny_df_cleaned.groupBy('PERP_SEX').count().show()


+--------+-------+
|PERP_SEX|  count|
+--------+-------+
|       F| 841392|
|       M|4171564|
+--------+-------+



In [208]:
ny_df_cleaned.groupBy('PERP_RACE').count().show()

+--------------------+-------+
|           PERP_RACE|  count|
+--------------------+-------+
|               WHITE| 604892|
|               BLACK|2433510|
|AMERICAN INDIAN/A...|  11050|
|      BLACK HISPANIC| 403034|
|      WHITE HISPANIC|1303555|
|             UNKNOWN|  50347|
|               OTHER|   1363|
|ASIAN / PACIFIC I...| 205205|
+--------------------+-------+



-  Saving all the racial details to create a table for info on race

In [209]:
ny_df_cleaned.select('PERP_RACE').distinct().show()


+--------------------+
|           PERP_RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|      BLACK HISPANIC|
|      WHITE HISPANIC|
|             UNKNOWN|
|               OTHER|
|ASIAN / PACIFIC I...|
+--------------------+



In [210]:
ny_race = ny_df_cleaned.select('PERP_RACE').distinct().collect()

In [211]:
ny_df_cleaned.groupBy('OFNS_DESC','KY_CD').count().show(50)

+--------------------+-----+------+
|           OFNS_DESC|KY_CD| count|
+--------------------+-----+------+
|INTOXICATED/IMPAI...|  119| 10398|
|            ESCAPE 3|  237|   283|
|    OTHER STATE LAWS|  125| 14140|
|NEW YORK CITY HEA...|  676|   514|
|OTHER STATE LAWS ...|  364| 79190|
|INTOXICATED & IMP...|  347| 99338|
|OTHER STATE LAWS ...|  364|  6907|
|MURDER & NON-NEGL...|  101| 12212|
|CHILD ABANDONMENT...|  120|   474|
|PROSTITUTION & RE...|  115|  1705|
|CRIMINAL MISCHIEF...|  351|  5279|
|HOMICIDE-NEGLIGEN...|  102|   109|
|MISCELLANEOUS PEN...|  126|159497|
|OFFENSES AGAINST ...|  363|  2047|
|    OTHER STATE LAWS|  677|227104|
|OFFENSES RELATED ...|  345|  2684|
|OFFENSES INVOLVIN...|  358| 26146|
|          SEX CRIMES|  116| 10565|
|OFF. AGNST PUB OR...|  361| 10263|
|             FORGERY|  113| 82649|
|                null| null|  9029|
|   LOITERING,BEGGING|  575|     1|
|       GRAND LARCENY|  109|119445|
|AGRICULTURE & MRK...|  364|    89|
|OTHER TRAFFIC INF...|  881|

In [212]:
ny_df_cleaned.groupBy('OFNS_DESC').count().show()

+--------------------+------+
|           OFNS_DESC| count|
+--------------------+------+
|OTHER TRAFFIC INF...|174429|
|ANTICIPATORY OFFE...|   703|
|HOMICIDE-NEGLIGEN...|   165|
|CHILD ABANDONMENT...|   474|
|NEW YORK CITY HEA...|   514|
|POSSESSION OF STO...|149877|
|OTHER OFFENSES RE...|  4262|
|VEHICLE AND TRAFF...|200292|
|            ABORTION|    10|
|KIDNAPPING & RELA...|  2026|
|HOMICIDE-NEGLIGEN...|   109|
|OFF. AGNST PUB OR...| 10263|
|      FELONY ASSAULT|196758|
|OFFENSES RELATED ...|  2684|
|ALCOHOLIC BEVERAG...|  5688|
|CRIMINAL MISCHIEF...| 11765|
|UNLAWFUL POSS. WE...|    63|
|         THEFT-FRAUD|  9127|
|   THEFT OF SERVICES|   479|
|                null|  9029|
+--------------------+------+
only showing top 20 rows



- I will create use this dataset to extract offense description and category key

In [213]:
ny_df_cleaned.groupBy('PD_DESC','PD_CD').count().show(50)

+--------------------+-----+------+
|             PD_DESC|PD_CD| count|
+--------------------+-----+------+
|US CODE,UNCLASSIFIED|   49| 10079|
|PROSTITUTION 2, U...|  588|    85|
|STOLEN PROPERTY-M...|  493|   158|
|  CRIMINAL MIS 2 & 3|  268|  3283|
|NY STATE LAWS,UNC...|  849|225678|
|FALSE REPORT UNCL...|  649|  5461|
|IMPERSONATION 2, ...|  707|   762|
|FOLLOWING TOO CLO...|  933|    24|
|             TAX LAW|  808|  3177|
|      SIGNAL,FAIL TO|  961|   549|
|FORGERY,ETC.,UNCL...|  729|  4712|
|LEAVING SCENE-ACC...|  940|  1481|
|SOLICITATION 5,CR...|  281|    23|
|MISCHIEF, CRIMINA...|  267|  1149|
|ASSAULT 2,1,UNCLA...|  109|162321|
|FUGITIVE,FROM OTH...|   15|   540|
|                null|  513|   149|
|VEHICULAR ASSAULT...|  104|   192|
|ROBBERY,UNCLASSIF...|  397|143263|
|             TAX LAW|  809| 13345|
|COMPUTER UNAUTH. ...|  770|    43|
|POSS METH MANUFAC...|  513|    17|
|CONTROLLED SUBSTA...|  511|278083|
|FRAUD,UNCLASSIFIE...|  739|  4054|
|THEFT,RELATED OFF...|  489|

- I will create use this dataset to extract offense description and category key

In [214]:
ny_df_cleaned.printSchema()

root
 |-- ARREST_KEY: integer (nullable = true)
 |-- arrest_date: date (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: integer (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- X_COORD_CD: double (nullable = true)
 |-- Y_COORD_CD: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lon_Lat: string (nullable = true)



- Rename some columns

In [215]:
ny_df_cleaned = ny_df_cleaned.withColumnRenamed("ARREST_KEY","arrest_key")\
.withColumnRenamed("PD_CD","pd_code")\
.withColumnRenamed("PD_DESC","pd_desc")\
.withColumnRenamed("KY_CD","offense_code")\
.withColumnRenamed("OFNS_DESC","offense_desc")\
.withColumnRenamed("LAW_CODE","law_code")\
.withColumnRenamed("LAW_CAT_CD","law_cat_code")\
.withColumnRenamed("ARREST_BORO","arrest_borough")\
.withColumnRenamed("ARREST_PRECINCT","arrest_precinct")\
.withColumnRenamed("JURISDICTION_CODE","jurisdiction_code")\
.withColumnRenamed("AGE_GROUP","age_group")\
.withColumnRenamed("PERP_SEX","perp_gender")\
.withColumnRenamed("per_race","perp_race")\
.withColumnRenamed("X_COORD_CD","x_coord_code")\
.withColumnRenamed("Y_COORD_CD","y_coord_code")\
.withColumnRenamed("Latitude","latitude")\
.withColumnRenamed("Longitude","longitude")\
.withColumnRenamed("Lon_Lat","lon_lat")

In [216]:
ny_df_cleaned.printSchema()

root
 |-- arrest_key: integer (nullable = true)
 |-- arrest_date: date (nullable = true)
 |-- pd_code: integer (nullable = true)
 |-- pd_desc: string (nullable = true)
 |-- offense_code: integer (nullable = true)
 |-- offense_desc: string (nullable = true)
 |-- law_code: string (nullable = true)
 |-- law_cat_code: string (nullable = true)
 |-- arrest_borough: string (nullable = true)
 |-- arrest_precinct: integer (nullable = true)
 |-- jurisdiction_code: integer (nullable = true)
 |-- age_group: string (nullable = true)
 |-- perp_gender: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- x_coord_code: double (nullable = true)
 |-- y_coord_code: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- lon_lat: string (nullable = true)



In [217]:
ny_df_cleaned.write.parquet("s3a://datalake-blm/stage/ny_crimes",mode="overwrite")


- I want to compare the names of race in NY crime dataset to the police death dataset

In [218]:
ny_race

[Row(PERP_RACE='WHITE'),
 Row(PERP_RACE='BLACK'),
 Row(PERP_RACE='AMERICAN INDIAN/ALASKAN NATIVE'),
 Row(PERP_RACE='BLACK HISPANIC'),
 Row(PERP_RACE='WHITE HISPANIC'),
 Row(PERP_RACE='UNKNOWN'),
 Row(PERP_RACE='OTHER'),
 Row(PERP_RACE='ASIAN / PACIFIC ISLANDER')]

In [219]:
race_list

array(['White', 'Unknown Race', 'Black', 'Hispanic', 'Pacific Islander',
       'Asian', 'Native American'], dtype=object)

- After looking at these two sets of data, I decided against grouping them up because I do not want to truncate granular information in the NY dataset. However, I will use the NY racial info to create a dimensional dataset


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [220]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [221]:
# Perform quality checks here